In [1]:
import os
import rasterio
from rasterio.windows import Window

def radiometric_correction(data):
    """
    Applies radiometric correction to each band with different factors.
    
    Parameters:
    - data: The image data as a numpy array.
    
    Returns:
    - corrected_data: Radiometrically corrected data.
    """
    correction_factors = [1.5242422, 1.8856289, 1.6708838, 1.1512102, 2.0653462]  # Example correction factors for each band
    corrected_data = data.copy()
    
    for i in range(data.shape[0]):
        corrected_data[i] /= correction_factors[i]
    
    return corrected_data

def clip_and_save_raster(file_path, output_folder, clip_size):
    """
    Clips a grid from the raster, applies radiometric correction, and saves it as a TIFF.
    
    Parameters:
    - file_path: Path to the input raster file.
    - output_folder: Folder where the corrected and clipped rasters will be saved.
    - clip_size: The size of the grid to be clipped (width, height).
    """
    with rasterio.open(file_path) as src:
        # Radiometric correction
        data = src.read().astype(float)
        corrected_data = radiometric_correction(data)
        
        # Define the corrected output path
        corrected_output_file_name = os.path.splitext(os.path.basename(file_path))[0] + '_corrected.tif'
        corrected_output_file_path = os.path.join(output_folder, corrected_output_file_name)
        
        # Save the corrected data to a new TIFF file with full size
        with rasterio.open(
            corrected_output_file_path,
            'w',
            driver='GTiff',
            count=src.count,
            dtype=corrected_data.dtype,
            width=src.width,
            height=src.height
        ) as dst:
            dst.write(corrected_data)
        
        # Calculate the window position (centered on the image) for clipping
        start_x = (src.width - clip_size[0]) // 2
        start_y = (src.height - clip_size[1]) // 2
        
        # Create a window and read the data for clipping
        window = Window(start_x, start_y, clip_size[0], clip_size[1])
        clipped_data = corrected_data[:, start_y:start_y + clip_size[1], start_x:start_x + clip_size[0]]
        
        # Save the clipped data to a new TIFF file
        clipped_output_file_name = os.path.splitext(os.path.basename(file_path))[0] + '_clipped.tif'
        clipped_output_file_path = os.path.join(output_folder, clipped_output_file_name)
        
        with rasterio.open(
            clipped_output_file_path,
            'w',
            driver='GTiff',
            count=src.count,
            dtype=clipped_data.dtype,
            width=clip_size[0],
            height=clip_size[1]
        ) as dst:
            dst.write(clipped_data)

def process_file(file_path, output_folder, clip_size):
    """
    Processes a single TIFF image, clipping and saving it as a new TIFF.
    
    Parameters:
    - file_path: Path to the TIFF image.
    - output_folder: Path to the folder where the clipped image will be saved.
    - clip_size: Dimensions of the clipped area.
    """
    # Ensure the output directory exists
    os.makedirs(output_folder, exist_ok=True)
    
    # Process the file
    clip_and_save_raster(file_path, output_folder, clip_size)

def process_folder(input_folder, output_folder, clip_size):
    """
    Processes all TIFF images in the specified input folder.
    
    Parameters:
    - input_folder: Path to the folder containing TIFF images.
    - output_folder: Path to the folder where the processed images will be saved.
    - clip_size: Dimensions of the clipped area.
    """
    for file_name in os.listdir(input_folder):
        if file_name.endswith('.tif'):
            file_path = os.path.join(input_folder, file_name)
            process_file(file_path, output_folder, clip_size)

# Example usage
input_folder = '/home/momty/HELIOS_new_manuscript_output/Biochemical/images/stacked_original_im/'  # Update this to your input folder
output_folder = '/home/momty/HELIOS_new_manuscript_output/Biochemical/images/stackaed_clipped_corrected/'  # Update this to your desired output folder
clip_size = (71, 103)  # Update this to the desired clipped size
process_folder(input_folder, output_folder, clip_size)


/home/momty/miniconda3/envs/vo/lib/python3.9/site-packages/rasterio/__init__.py:220: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
/home/momty/miniconda3/envs/vo/lib/python3.9/site-packages/rasterio/__init__.py:230: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix be returned.
  s = writer(path, mode, driver=driver,
